<a href="https://colab.research.google.com/github/muzakkialfarisi/MACHINE-LEARNING/blob/master/TfIDF_PPMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import library python**

In [ ]:
import nltk, re, pprint, random, string, heapq
from nltk import word_tokenize # tokenisasi
nltk.download('punkt') #
import numpy as np # mengakses numpy
import math # mengakses formula pada python

from google.colab import drive
drive.mount('/content/drive/') # untuk dapat mengakses file / folder didalam drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Corpus**

In [ ]:
def set_corpus(artikel): # fungsi membuat corpus
  corpus = nltk.sent_tokenize(artikel)
  for i in range(len(corpus)):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i]) # menghapus tanda baca
    corpus [i] = re.sub(r'\s+',' ',corpus [i]) # menghapus spasi berlebih
  return corpus

**Hitung Frequensi**

In [ ]:
def hitfreq(corpus): # fungsi menhitung frequensi dengan parameter crpus
  wordfreq = {}
  for sentence in corpus:
      tokens = nltk.word_tokenize(sentence)
      for token in tokens:
          if token not in wordfreq.keys():
              wordfreq[token] = 1
          else:
              wordfreq[token] += 1
  return wordfreq

**TF-IDF**

In [ ]:
def mostfreq(wordfreq): # fungsi untuk set hitung frequensi maksimum
  most_freq = heapq.nlargest(100, wordfreq, key=wordfreq.get) # setting wordfreq bisa diubah sesuai dengan karakteristik data yang digunakan.
  return most_freq

In [ ]:
def idf_value(corpus): # fungsi menghitung nilai idf
  most_freq = mostfreq(hitfreq(corpus)) # memanggil fungsi set frequensi maksimum dengan parameter fungsi mengitung frequensi
  word_idf_values = {}
  for token in most_freq:
      doc_containing_word = 0
      for document in corpus:
          if token in nltk.word_tokenize(document):
              doc_containing_word += 1
      word_idf_values[token] = np.log10(len(corpus)/(doc_containing_word))
  return word_idf_values

In [ ]:
def tf_value(corpus): # fungsi mengitung nilai tf
  most_freq = mostfreq(hitfreq(corpus)) # memanggil fungsi set frequensi maksimum dengan parameter fungsi mengitung frequensi
  word_tf_values = {}
  for token in most_freq:
      sent_tf_vector = []
      for document in corpus:
          doc_freq = 0
          for word in nltk.word_tokenize(document):
              if token == word:
                    doc_freq += 1
          # word_tf = doc_freq # tanpa normalisasi
          word_tf = doc_freq/len(nltk.word_tokenize(document)) # normalisasi dengan panjang dokumen
          sent_tf_vector.append(word_tf)
      word_tf_values[token] = sent_tf_vector
  return word_tf_values

In [ ]:
 def tfidf_value(corpus): # fungsi mengitung nilai tfidf
  word_tf_values = tf_value(corpus) # memanggil fungsi menghitung nilai tf
  word_idf_values = idf_value(corpus) # memanggil fungsi menghitung nilai idf
  tfidf_values = []
  for token in word_tf_values.keys():
      tfidf_sentences = [] # 1 dokumen adalah 1 kalimat
      for tf_sentence in word_tf_values[token]:
          tf_idf_score = tf_sentence * word_idf_values[token]
          tfidf_sentences.append(tf_idf_score)
      tfidf_values.append(tfidf_sentences)
  return tfidf_values

In [ ]:
def not_0_tfidf(tfidf_values): # fungsi mencari nilai tidak sama dengan 0 dengan parameter return nilai tfidf
  pembilang = 0 
  penyebut = 0
  for i in range(len(tfidf_values)):
    for j in tfidf_values[i]:
      if j != 0:
        pembilang += 1
      else:
        pembilang = pembilang
      penyebut += 1
  return pembilang/penyebut*100

In [ ]:
def cosine_similarity_sama(tfidf_values, x, y): # fungsi menghitung cosine_similarity kalimat dengan topik yang sama
  tf_idf_model = np.asarray(tfidf_values)
  tf_idf_model = np.transpose(tf_idf_model)
  if (x < len(tf_idf_model) and y < len(tf_idf_model)):
    return np.dot(tf_idf_model[x],tf_idf_model[y])/np.linalg.norm(tf_idf_model[x])*np.linalg.norm(tf_idf_model[y])
  else:
    return False

In [ ]:
def cosine_similarity_beda(tfidf_values1, tfidf_values2, x, y): # fungsi menghitung cosine_similarity kalimat dengan topik yang berbeda
  tf_idf_model1 = np.asarray(tfidf_values1)
  tf_idf_model1 = np.transpose(tf_idf_model1)
  tf_idf_model2 = np.asarray(tfidf_values2)
  tf_idf_model2 = np.transpose(tf_idf_model2)
  return np.dot(tf_idf_model1[x],tf_idf_model2[y])/np.linalg.norm(tf_idf_model1[x])*np.linalg.norm(tf_idf_model2[y])

In [ ]:
def kata_cosine_similarity_sama(tfidf_values, x, y): # fungsi menghitung cosine_similarity kata dengan topik yang sama
  tf_idf_model = np.asarray(tfidf_values)
  if (x < len(tf_idf_model) and y < len(tf_idf_model)):
    return np.dot(tf_idf_model[x],tf_idf_model[y])/np.linalg.norm(tf_idf_model[x])*np.linalg.norm(tf_idf_model[y])
  else:
    return False

In [ ]:
def kata_cosine_similarity_beda(tfidf_values1, tfidf_values2, x, y): # fungsi menghitung cosine_similarity kata dengan topik yang berbeda
  # karena ordonya berbeda, maka samakan ordo dgn meng sign angka 0 pada new pixel
  if len(tfidf_values1[0]) < len(tfidf_values2[0]):
    baru = tfidf_values1
    for i in range(len(tfidf_values1)):
      for j in range(len(tfidf_values2[i])-len(tfidf_values1[i])):
        baru[i].append(0)
    tf_idf_model1 = np.asarray(baru)
    tf_idf_model2 = np.asarray(tfidf_values2)
  else:
    baru = tfidf_values2
    for i in range(len(tfidf_values1)):
      for j in range(len(tfidf_values1[i])-len(tfidf_values2[i])):
        baru[i].append(0)
    tf_idf_model1 = np.asarray(tfidf_values1)
    tf_idf_model2 = np.asarray(baru)
  
  return np.dot(tf_idf_model1[x],tf_idf_model2[y])/np.linalg.norm(tf_idf_model1[x])*np.linalg.norm(tf_idf_model2[y])

**PPMI**

In [ ]:
# inisialisasi matriks co-occurrence
def initial_matriks_co(vocab_freq):
  co_occurrence_mat = {}
  for token_1 in vocab_freq.keys():
    co_occurrence_terms = []
    for token_2 in vocab_freq.keys():
      co_occurrence_mat[(token_1,token_2)] = 0
  return co_occurrence_mat

In [ ]:
def proses_matriks_co(corpus, co_occurrence_mat):
  window_size = 2 # contoh, ukuran window = 2
  # proses
  for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for i in range(0,len(tokens)):
      # konteks kata-kata sebelah kiri
      left_index = i-1
      while left_index >= 0 and left_index >= i-window_size:
        token_1 = tokens[i]
        token_2 = tokens[left_index]
        co_occurrence_mat[(token_1,token_2)] += 1
        left_index = left_index - 1
      # konteks kata-kata sebelah kanan
      right_index = i+1
      while right_index < len(tokens) and right_index <= i+window_size:
        token_1 = tokens[i]
        token_2 = tokens[right_index] 
        co_occurrence_mat[(token_1,token_2)] += 1    
        right_index = right_index + 1
  return co_occurrence_mat

In [ ]:
def cooccurrence_mat(wordfreq, co_occurrence_mat):
  bigram_count = {}
  str_token = '\t'
  for token in wordfreq.keys():
    str_token += '\t\t'+token
  for token_1 in wordfreq.keys():
    str_row = token_1+'\t'
    curr_bigram_count = 0
    bigram_count[token_1] = 0
    for token_2 in wordfreq.keys():
      str_row += '\t\t'+str(co_occurrence_mat[(token_1,token_2)])
      curr_bigram_count += co_occurrence_mat[(token_1,token_2)] # update jumlah kemunculan term
    bigram_count[token_1] = curr_bigram_count # assignemnt jumlah kemunculan term sebagai bagian dari bigram
  return bigram_count

In [ ]:
def prob_term(wordfreq, co_occurrence_mat):
  term_context_prob = {}
  sum_term_context = sum(co_occurrence_mat[item] for item in co_occurrence_mat)
  for token_1 in wordfreq.keys():
    for token_2 in wordfreq.keys():    
      term_context_prob[(token_1,token_2)] = co_occurrence_mat[(token_1,token_2)]/sum_term_context
  return term_context_prob

In [ ]:
def pmi(wordfreq, co_occurrence_mat, bigram_count, term_context_prob):
  pmi_mat = {}
  sum_term_context = sum(co_occurrence_mat[item] for item in co_occurrence_mat)
  for token_1 in wordfreq.keys():
    for token_2 in wordfreq.keys(): 
      token_1_prob = bigram_count[token_1]/sum_term_context
      token_2_prob = bigram_count[token_2]/sum_term_context
      if term_context_prob[(token_1,token_2)] > 0:
        pmi_mat[(token_1,token_2)] = round(math.log2(term_context_prob[(token_1,token_2)]/(token_1_prob*token_2_prob)), 4)
      else: # kalau nilai probability = 0, tidak bisa dihitung log 2 -nya
        pmi_mat[(token_1,token_2)] = None
  return pmi_mat

In [ ]:
def not_0_ppmi(wordfreq, pmi_mat):
  pembilang = 0
  penyebut = 0
  for token_1 in wordfreq.keys():
    for token_2 in wordfreq.keys():
      if pmi_mat[(token_1,token_2)] is None or pmi_mat[(token_1,token_2)] > 0:
        pembilang += 1
      penyebut += 1
  return (pembilang/penyebut*100)

**MAIN**

In [ ]:
t_1 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/a1.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a2.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a3.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a4.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a5.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a6.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a7.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a8.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a9.txt').read()
t_1 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/a10.txt').read()
t_2 = open('/content/drive/My Drive/Colab Notebooks/DataTrain/b1.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b2.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b3.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b4.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b5.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b6.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b7.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b8.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b9.txt').read()
t_2 += open('/content/drive/My Drive/Colab Notebooks/DataTrain/b10.txt').read()
article = t_1 + t_2

In [ ]:
corpus1 = set_corpus(t_1) # fungsi pembuatan corpus dengan topik A
wordfreq1 = hitfreq(corpus1) # fungsi hitung frequensi corpus topik A

corpus2 = set_corpus(t_2) # fungsi pembuatan corpus dengan topik B
wordfreq2 = hitfreq(corpus2) # fungsi hitung frequensi corpus topik B

**PPMI**

In [ ]:
# Topik A
co_occurrence_mat1 = initial_matriks_co(wordfreq1)
co_occurrence_mat1 = proses_matriks_co(corpus1, co_occurrence_mat1)
bigram_count1 = cooccurrence_mat(wordfreq1, co_occurrence_mat1)
term_context_prob1 = prob_term(wordfreq1, co_occurrence_mat1)
pmi_mat1 = pmi(wordfreq1, co_occurrence_mat1, bigram_count1, term_context_prob1)

# Topik B
co_occurrence_mat2 = initial_matriks_co(wordfreq2)
co_occurrence_mat2 = proses_matriks_co(corpus2, co_occurrence_mat2)
bigram_count2 = cooccurrence_mat(wordfreq2, co_occurrence_mat2)
term_context_prob2 = prob_term(wordfreq2, co_occurrence_mat2)
pmi_mat2 = pmi(wordfreq2, co_occurrence_mat2, bigram_count2, term_context_prob2)

In [ ]:
print('Elemen tidak sama dengan nol PPMI')
print('Topik A :',not_0_ppmi(wordfreq1, pmi_mat1),'%')
print('Topik B :',not_0_ppmi(wordfreq2, pmi_mat2),'% ')

Elemen tidak sama dengan nol PPMI
Topik A : 99.98659279778394 %
Topik B : 99.9952844959384 % 


**TF-IDF**

In [ ]:
# Topik A
word_idf_values1 = idf_value(corpus1)
word_tf_values1 = tf_value(corpus1)
tfidf_values1 = tfidf_value(corpus1)
not_0_1 = not_0_tfidf(tfidf_values1)

# Topik B
word_idf_values2 = idf_value(corpus2)
word_tf_values2 = tf_value(corpus2)
tfidf_values2 = tfidf_value(corpus2)
not_0_2 = not_0_tfidf(tfidf_values2)

In [ ]:
print('Elemen tidak sama dengan nol TF-IDF')
print('Topik-A   :',not_0_1,'%')
print('Topik-B   :',not_0_2,'%')

Elemen tidak sama dengan nol TF-IDF
Topik-A   : 9.097222222222221 %
Topik-B   : 7.1878787878787875 %


In [ ]:
print('Cosine Similarity kata dengan topik yang sama')
print('Topik A dengan panjang 0 sampai dengan',len(tfidf_values1[0])-1)
kal_a_1 = 10
kal_a_2 = 7
print('- Kalimat 10 dan 7 :',cosine_similarity_sama(tfidf_values1,kal_a_1,kal_a_2))
print('Topik B dengan panjang 0 sampai dengan',len(tfidf_values2[0])-1)
kal_b_1 = 10
kal_b_2 = 7
print('- Kalimat 10 dan 7 :',cosine_similarity_sama(tfidf_values2,kal_b_1,kal_b_2))

Cosine Similarity kata dengan topik yang sama
Topik A dengan panjang 0 sampai dengan 143
- Kalimat 10 dan 7 : 0.005213896668408891
Topik B dengan panjang 0 sampai dengan 164
- Kalimat 10 dan 7 : 0.02523579819493903


In [ ]:
print('Cosine Similarity kalimat dengan topik yang berbeda')
print('Topik A dengan panjang kalimat 0 sampai dengan',len(tfidf_values1[0])-1) 
print('Topik B dengan panjang kalimat 0 sampai dengan',len(tfidf_values2[0])-1)
kal_topik_a = 10
kal_topik_b = 7
print('- Topik A Kalimat-10 dan Topik B kalimat-7:',cosine_similarity_beda(tfidf_values1,tfidf_values2,kal_topik_a,kal_topik_b))

Cosine Similarity kalimat dengan topik yang berbeda
Topik A dengan panjang kalimat 0 sampai dengan 143
Topik B dengan panjang kalimat 0 sampai dengan 164
- Topik A Kalimat-10 dan Topik B kalimat-7: 0.0019532851562571734


In [ ]:
print('Cosine Similarity kata dengan topik yang sama')
print('Topik A dengan panjang 0 sampai dengan',len(tfidf_values1)-1)
kal_a_1 = 6
kal_a_2 = 4
print('- kata 0 dan 2 :',kata_cosine_similarity_sama(tfidf_values1,kal_a_1,kal_a_2))
print('Topik B dengan panjang 0 sampai dengan',len(tfidf_values2)-1)
kal_b_1 = 6
kal_b_2 = 4
print('- kata 2 dan 0 :',kata_cosine_similarity_sama(tfidf_values2,kal_b_1,kal_b_2))

Cosine Similarity kata dengan topik yang sama
Topik A dengan panjang 0 sampai dengan 99
- kata 0 dan 2 : 0.008678264583141861
Topik B dengan panjang 0 sampai dengan 99
- kata 2 dan 0 : 0.0036073884440326392


In [ ]:
print('Cosine Similarity kata dengan topik yang beda')
print('Topik A dengan panjang kalimat 0 sampai dengan',len(tfidf_values1)-1) 
print('Topik B dengan panjang kalimat 0 sampai dengan',len(tfidf_values2)-1)
kata_topik_a = 6
kata_topik_b = 4
print('- Topik A kata-6 dan Topik B kata-4:',kata_cosine_similarity_beda(tfidf_values1,tfidf_values2,kata_topik_a,kata_topik_b))

Cosine Similarity kata dengan topik yang beda
Topik A dengan panjang kalimat 0 sampai dengan 99
Topik B dengan panjang kalimat 0 sampai dengan 99
- Topik A kata-6 dan Topik B kata-4: 0.0038319178720998804
